In [1]:
!pip install svgpath2mpl

In [2]:
from collections import namedtuple, deque
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adamax
import random
from svgpath2mpl import parse_path
import matplotlib.pyplot as plt
import matplotlib
from scipy.ndimage import rotate, shift
from matplotlib.animation import FuncAnimation
from probabilistic_fire_env import ProbabilisticFireEnv
from drone_env import DronesEnv
from replay_memory import Transition
from models.drqn import DRQN
from episode_buffer import EpisodeBuffer

In [4]:
height = width = 100
BATCH_SIZE = 5
GAMMA = 0.99
INIT_SIZE = 10
TARGET_UPDATE = 1000
SAVE_POLICY = 100
EPISODE_LENGTH = 250
TRAIN_FREQ  = 10   # Number of samples to generate between trainings (Should be multiple of 10)
PRINT_FREQ  = 100  # Frequency of printing (Should be a multiple of 10)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_actions = 2
screen_height = screen_width = 100
channels = 2
policy_net = DRQN(device, channels, screen_height, screen_width, n_actions).to(device)
target_net = DRQN(device, channels, screen_height, screen_width, n_actions).to(device)
steps = 0
target_net.load_state_dict(policy_net.state_dict())
episode_buffer = EpisodeBuffer()
policy_net.train()
target_net.eval()
episode_length = 128
update_counter = 0
optimizer = Adamax(policy_net.parameters(), lr=0.0001)

In [6]:
def optimize_model():
    
    loss = None
    
    global update_counter
    update_counter += 1

    episode_batches, epiosde_length = episode_buffer.sample(BATCH_SIZE, episode_length)
    
    for episode_batch in episode_batches:


        batch = Transition(*zip(*episode_batch))

        next_states = torch.cat(batch.next_state_vector)
        next_belief_map = torch.cat(batch.next_belief_map)

        belief_map_batch = torch.cat(batch.belief_map)
        state_vector_batch = torch.cat(batch.state_vector)
    
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)

        hidden_policy = policy_net.init_hidden_state()
        hidden_target = target_net.init_hidden_state()

        policy_output, _ = policy_net(belief_map_batch, state_vector_batch, hidden_policy)
        target_output, _ = target_net(next_belief_map, next_states, hidden_target)

        state_action_values = policy_output.gather(1, action_batch)
        next_state_values = target_output.max(1)[0].detach()

        expected_state_action_values = (next_state_values * GAMMA) + reward_batch

        # Compute Huber loss
        criterion = nn.SmoothL1Loss().to(device)
        loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        for param in policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer.step()

        if update_counter % TARGET_UPDATE == 0:
            policy_file_path = f'./policy_weights_drqn.pt'
            target_file_path = f'./target_weights_drqn.pt'
            torch.save(policy_net.state_dict(), policy_file_path)
            torch.save(target_net.state_dict(), target_file_path)
            print('update target')
            target_net.load_state_dict(policy_net.state_dict())

    return loss

In [7]:
DT          = 0.5  # Time between wildfire updates            
DTI         = 0.1  # Time between aircraft decisions
fireEnv = ProbabilisticFireEnv(height, width)
dronesEnv = DronesEnv(height, width, DT, DTI) 
loss = None
i_episode = 1

observation = fireEnv.reset()
dronesEnv.reset(observation)

episode_memory_1 = []
episode_memory_2 = []

hidden_1 = policy_net.init_hidden_state()
hidden_2 = policy_net.init_hidden_state()

while True:
  # Initialize the environment and state
  #env.reset()
  for j in range(TRAIN_FREQ//int(2*DT/DTI)):

    observation = fireEnv.step()

    state_vector_1 = dronesEnv.drones[0].state
    map_1 = dronesEnv.drones[0].observation
    state_vector_1 = torch.tensor(state_vector_1, device=device, dtype=torch.float)
    map_1 = torch.tensor(map_1, device=device, dtype=torch.float)

    state_vector_2 = dronesEnv.drones[1].state
    map_2 = dronesEnv.drones[1].observation
    state_vector_2 = torch.tensor(state_vector_2, device=device, dtype=torch.float)
    map_2 = torch.tensor(map_2, device=device, dtype=torch.float)


    for i in range(int(DT/DTI)):

      action1, hidden_1 = policy_net.select_action(map_1, state_vector_1, steps, hidden=hidden_1)
      action2, hidden_2 = policy_net.select_action(map_2, state_vector_2, steps, hidden=hidden_2)
      steps += 2
      reward_1, reward_2 = dronesEnv.step([action1.item(), action2.item()], observation)

      next_state_vector_1 = dronesEnv.drones[0].state
      next_map_1 = dronesEnv.drones[0].observation

      next_state_vector_1 = torch.tensor(next_state_vector_1, device=device, dtype=torch.float)
      next_map_1 = torch.tensor(next_map_1, device=device, dtype=torch.float)

      next_state_vector_2 = dronesEnv.drones[1].state
      next_map_2 = dronesEnv.drones[1].observation

      next_state_vector_2 = torch.tensor(next_state_vector_2, device=device, dtype=torch.float)
      next_map_2 = torch.tensor(next_map_2, device=device, dtype=torch.float)

      reward_1 = torch.tensor([reward_1], device=device)
      reward_2 = torch.tensor([reward_2], device=device)  

      episode_memory_1.append(Transition(map_1, state_vector_1, action1, next_map_1, next_state_vector_1, reward_1))
      episode_memory_2.append(Transition(map_2, state_vector_2, action2, next_map_2, next_state_vector_2, reward_2))

      state_vector_1 = next_state_vector_1
      state_vector_2 = next_state_vector_2

      map_1 = next_map_1
      map_2 = next_map_2


    if not fireEnv.fire_in_range(6):
      observation = fireEnv.reset()
      dronesEnv.reset(observation)
      episode_buffer.push(episode_memory_1.copy())
      episode_buffer.push(episode_memory_2.copy())
      episode_memory_1 = []
      episode_memory_2 = []
      hidden_1 = policy_net.init_hidden_state()
      hidden_2 = policy_net.init_hidden_state()
      i_episode +=1
      
      if (i_episode+1) % 5 == 0:
        print(f'{i_episode+1} episodes completed')
        print(f'loss {loss}')
        print(f'steps done {steps}')

      if i_episode>=INIT_SIZE:
        loss = optimize_model()

5 episodes completed
loss None
steps done 2140
10 episodes completed
loss None
steps done 5270
15 episodes completed
loss 1.5806398391723633
steps done 9460
20 episodes completed
loss 0.5534675121307373
steps done 13470
25 episodes completed
loss 1.4399771690368652
steps done 17120
30 episodes completed
loss 0.012638617306947708
steps done 20930
35 episodes completed
loss 0.01372477412223816
steps done 24790
40 episodes completed
loss 1.4912450313568115
steps done 29080
45 episodes completed
loss 0.003751372918486595
steps done 33690
50 episodes completed
loss 0.002313715871423483
steps done 37540
55 episodes completed
loss 0.004201896488666534
steps done 41180
60 episodes completed
loss 0.40921664237976074
steps done 45180
65 episodes completed
loss 0.016253996640443802
steps done 49200
70 episodes completed
loss 0.011843782849609852
steps done 53410
75 episodes completed
loss 2.1556262969970703
steps done 56630
80 episodes completed
loss 2.9586000442504883
steps done 60030
85 episode

: 